In [1]:
import pandas as pd
import sqlite3

## Создаем подключение к базе данных

In [2]:
connection = sqlite3.connect('data/checking-logs.sqlite')

## Создайте два фрейма данных: test_results и control_results со столбцами time и avg_diff и только двумя строками
- время должно иметь значения: after и before
- avg_diff содержит среднюю дельту среди всех пользователей за период времени до того, как каждый из них совершил свой первый визит на страницу, и после этого
- учитывать только тех пользователей, у которых есть наблюдения до и после

In [3]:
query_test = '''
SELECT 
    'after' AS time,
    AVG(CAST((julianday(first_commit_ts) - julianday(first_view_ts)) AS REAL)) AS avg_diff
FROM test
WHERE first_commit_ts IS NOT NULL AND first_view_ts IS NOT NULL
    AND julianday(first_commit_ts) > julianday(first_view_ts)
UNION ALL
SELECT 
    'before' AS time,
    AVG(CAST((julianday(first_view_ts) - julianday(first_commit_ts)) AS REAL)) AS avg_diff
FROM test
WHERE first_commit_ts IS NOT NULL AND first_view_ts IS NOT NULL
    AND julianday(first_view_ts) > julianday(first_commit_ts);
'''

test_results = pd.io.sql.read_sql(query_test, connection)

query_control = '''
SELECT 
    'after' AS time,
    AVG(CAST((julianday(first_commit_ts) - julianday(first_view_ts)) AS REAL)) AS avg_diff
FROM control
WHERE first_commit_ts IS NOT NULL AND first_view_ts IS NOT NULL
  AND julianday(first_commit_ts) > julianday(first_view_ts)
UNION ALL
SELECT 
    'before' AS time,
    AVG(CAST((julianday(first_view_ts) - julianday(first_commit_ts)) AS REAL)) AS avg_diff
FROM control
WHERE first_commit_ts IS NOT NULL AND first_view_ts IS NOT NULL
  AND julianday(first_view_ts) > julianday(first_commit_ts);
'''

control_results = pd.io.sql.read_sql(query_control, connection)
control_results

,time,avg_diff
0,after,14.263309
1,before,4.384062


## Закрыть соединение

In [4]:
connection.close()

гипотеза оказалась верной и страница действительно влияет на поведение студентов?

In [5]:
print('Test Results:')
print(test_results)

print('\nControl Results:')
print(control_results)

if test_results.loc[
    test_results['time'] == 'after', 'avg_diff'
    ].values[0] > test_results.loc[
        test_results['time'] == 'before', 'avg_diff'
        ].values[0]:
    print('\nГипотеза подтвердилась: страница действительно влияет на поведение студентов.')
else:
    print('\nГипотеза не подтвердилась: страница не влияет на поведение студентов.')

Test Results:
     time   avg_diff
0   after  14.509470
1  before   6.498336

Control Results:
     time   avg_diff
0   after  14.263309
1  before   4.384062

Гипотеза подтвердилась: страница действительно влияет на поведение студентов.
